In [ ]:
!pip install transformers peft datasets accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
!pip install -qU "transformers>=4.41.0" accelerate bitsandbytes \
               datasets sentencepiece peft wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 136.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, gc, torch, bitsandbytes
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
)
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Digits
from peft import (
    LoraConfig, get_peft_model, prepare_model_for_kbit_training,
    AutoPeftModel
)
from peft.optimizers import create_loraplus_optimizer
import wandb, huggingface_hub

In [ ]:
CLRS_TEXT_FIELDS = ["question", "answer", "algo_name"]
GEMMA_MODEL_NAME = "google/gemma-2b"

HF_TOKEN   = "redacted"
WANDB_KEY  = "redacted"
PROJECT    = "NLP_PROJECT_GEMMA2B"

os.environ["WANDB_PROJECT"] = PROJECT
os.environ["WANDB_API_KEY"] = WANDB_KEY
wandb.login(key=WANDB_KEY)
huggingface_hub.login(token=HF_TOKEN)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: m-wysoglad (m-wysoglad-university-of-warsaw) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
TOKENIZER_DIR = "/content/drive/MyDrive/NLP_ZOFIA/gemma_fine_tuned/digits_and_diff"
MODEL_DIR = "/content/drive/MyDrive/NLP_ZOFIA/gemma_fine_tuned/full"

In [ ]:
from huggingface_hub import login
login()

In [ ]:
def load_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(GEMMA_MODEL_NAME)

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    if tokenizer.chat_template is None:
        tokenizer.chat_template = (
            "{% for message in messages %}"
            "{% if message['role'] == 'user' %}"
            "{{ '<|start_header_id|>user<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}"
            "{% elif message['role'] == 'assistant' %}"
            "{{ '<|start_header_id|>assistant<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}"
            "{% endif %}"
            "{% endfor %}"
        )
    return tokenizer


In [ ]:
def remove_special_characters(text):
    return text.replace("_", " ").replace("\n", " ")

def format_example_initial(example):
    # This function is used to prepare the 'text' and 'out' columns
    # from your original 'question' and 'answer' fields.
    return {
        "text": remove_special_characters(example["question"]),
        "out": remove_special_characters(example["answer"]),
    }

def create_causal_lm_formatting_function(tokenizer, max_length=1024): # Increased max_length
    def _inner_format(example):
        question = remove_special_characters(example["question"])
        answer = remove_special_characters(example["answer"])


        messages_for_prompt = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": ""}
        ]


        prompt_tokens = tokenizer.apply_chat_template(
            messages_for_prompt,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            return_dict=True,
        )
        prompt_input_ids = prompt_tokens["input_ids"].squeeze()
        prompt_length = len(prompt_input_ids)

        messages_full_sequence = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]

        # Tokenize the full sequence
        full_tokenized = tokenizer.apply_chat_template(
            messages_full_sequence,
            tokenize=True,
            add_generation_prompt=False, # We want the full sequence, not an inference prompt
            max_length=max_length,
            truncation=True,
            padding="max_length", # Pad here, DataCollatorForLanguageModeling can then handle it
            return_tensors="pt",
            return_attention_mask=True, # Ensure we get attention mask
            return_dict = True
        )

        input_ids = full_tokenized["input_ids"].squeeze()
        attention_mask = full_tokenized["attention_mask"].squeeze()
        labels = input_ids.clone()

        # Step 3: Mask out the prompt part from labels so loss is only on the answer
        # Ensure prompt_length does not exceed the actual length of the tokenized sequence (due to truncation)
        actual_prompt_length = min(prompt_length, len(input_ids))
        labels[:actual_prompt_length] = -100 # Mask prompt tokens for loss calculation

        return {
            "input_ids": input_ids.tolist(), # Convert to list for dataset map
            "attention_mask": attention_mask.tolist(), # Convert to list for dataset map
            "labels": labels.tolist(), # Convert to list for dataset map
        }
    return _inner_format



In [ ]:
import pandas as pd
from datasets import Dataset

def df_to_dataset(df):
    return Dataset.from_pandas(df.reset_index(drop=True))  # drop the pd.Index col

train_df = pd.read_parquet(
    "/content/drive/MyDrive/NLP_ZOFIA/training_cached/train-00000-of-00005.parquet"
)
test_df = pd.read_parquet(
    "/content/drive/MyDrive/NLP_ZOFIA/cached_dataset/test_1-00000-of-00001.parquet"
)

train_dataset = df_to_dataset(train_df)
test_dataset  = df_to_dataset(test_df)


In [ ]:
print(train_dataset)

Dataset({
    features: ['question', 'answer', 'algo_name'],
    num_rows: 430000
})


In [ ]:
tokenizer = load_tokenizer()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
tok_train = train_dataset.map(
    create_causal_lm_formatting_function(tokenizer),
)

tok_test  = test_dataset.map(
    create_causal_lm_formatting_function(tokenizer),
)

Map:   0%|          | 0/430000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100400 [00:00<?, ? examples/s]

In [ ]:
def fine_tune_model(model_name, train_dataset, test_dataset, tokenizer_name, output_dir, memory_efficient=True, use_wandb=False, resume_from_checkpoint=None):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"


    if tokenizer.chat_template is None:
        tokenizer.chat_template = (
            "{% for message in messages %}"
            "{% if message['role'] == 'user' %}"
            "{{ '<|start_header_id|>user<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}"
            "{% elif message['role'] == 'assistant' %}"
            "{{ '<|start_header_id|>assistant<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}"
            "{% endif %}"
            "{% endfor %}"
        )

    tokenized_test_dataset = test_dataset.map(create_causal_lm_formatting_function(tokenizer), batched=False)
    tokenized_train_dataset = train_dataset.map(create_causal_lm_formatting_function(tokenizer), batched=False)

    print("Tokenized train dataset:", tokenized_train_dataset,tokenized_train_dataset[0] )
    print("Tokenized test dataset:", tokenized_test_dataset,tokenized_test_dataset[0])

    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True, device_map="auto", load_in_4bit=memory_efficient)
    model.resize_token_embeddings(len(tokenizer))

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    lora_config = LoraConfig(
        target_modules=["q_proj", "k_proj"],
        modules_to_save=["lm_head", "embed_tokens"], # Added "embed_tokens"
        init_lora_weights="gaussian",
    )

    model = get_peft_model(model, lora_config)

    optimizer = create_loraplus_optimizer(
        model=model,
        optimizer_cls=bitsandbytes.optim.Adam8bit,
        lr=5e-5,
        loraplus_lr_ratio=16,
    )
    scheduler = None

    prepare_model_for_kbit_training(model)


    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="steps",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=10,
        gradient_checkpointing=True,
        num_train_epochs=1,
        logging_steps=100,
        save_total_limit=3,
        push_to_hub=False,
        report_to="wandb" if use_wandb else "none",
        run_name="fine-tune-{}-model".format(model_name),
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        data_collator=data_collator,
        optimizers=(optimizer, scheduler)
    )

    print("Starting training...")

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    result = trainer.evaluate()
    print(f"Evaluation results: {result}")

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    del(model)
    del(tokenizer)
    del(trainer)

In [ ]:
fine_tune_model(
    model_name=GEMMA_MODEL_NAME,
    train_dataset=train_dataset.shuffle().select(range(2000)),
    test_dataset=test_dataset.shuffle().select(range(200)),
    tokenizer_name=GEMMA_MODEL_NAME,
    output_dir=MODEL_DIR,
    memory_efficient=True,
    use_wandb=True,
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenized train dataset: Dataset({
    features: ['question', 'answer', 'algo_name', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
}) {'question': 'topological_sort:\nA: [[0 0 1 1 0], [1 0 1 1 1], [0 0 0 1 0], [0 0 0 0 0], [0 0 1 1 0]], initial_trace: [0 1 2 3 4], 0\ntrace | topo, topo_head:\n', 'answer': '[0 1 2 3 4], 0, [0 1 2 3 4], 0, [0 1 2 3 4], 0, [0 1 2 3 4], 3, [0 1 3 3 4], 2, [2 1 3 3 4], 0, [2 1 3 3 4], 0, [2 1 3 3 4], 0, [2 1 3 3 4], 0, [2 1 3 3 0], 4 | [2 4 3 3 0], 1\n\n', 'algo_name': 'topological_sort', 'input_ids': [235322, 235371, 2997, 235298, 5351, 235298, 539, 73786, 1645, 235322, 235371, 615, 235298, 5351, 235298, 539, 73786, 108, 64567, 3676, 6728, 235292, 586, 235292, 15695, 235276, 235248, 235276, 235248, 235274, 235248, 235274, 235248, 235276, 1308, 892, 235274, 235248, 235276, 235248, 235274, 235248, 235274, 235248, 235274, 1308, 892, 235276, 235248, 235276, 235248, 235276, 235248, 235274, 235248, 235276, 1308, 892, 235276, 235248, 235276, 235248

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


Evaluation results: {'eval_runtime': 57.1008, 'eval_samples_per_second': 3.503, 'eval_steps_per_second': 0.438, 'epoch': 1.0}
